In [1]:
import pandas as pd
import numpy as np
from jinja2 import Template
import os, glob

from kompy import KomootConnector, Authentication
password = 'motorcyclE1'
email= "benhastings17+k@gmail.com"
connector = KomootConnector(password=password, email=email)
authenticator = Authentication(password=password, email_address=email)


def download_tour(komoot_url, authenticator=authenticator):
    tour_id = komoot_url.split('/tour/')[1].split('?ref')[0]
    tour = connector.get_tour_by_id(tour_identifier=tour_id)
    tour.generate_gpx_track(authenticator)
    return tour
    

def add_komoot_url_to_collection(komoot_url):
    tour = download_tour(komoot_url)

    # find name of new gpx
    gpxs = sorted(glob.glob('gpxs/*.gpx'))
    last_int = int(os.path.splitext(os.path.basename(gpxs[-1]))[0])
    i = last_int + 1
    
    d=({'i':str(i).zfill(3), 
       'name': tour.name.replace(" ", "").replace(".", "_"),
       'distance': int(tour.distance/1000), 
       'elevation': int(tour.elevation_up),
      'url': "https://rdspt.com/inlawsgravel/gpxs/%s.gpx"%str(i).zfill(3)})

    #save gpx file
    gpx = tour.gpx_track
    with open("gpxs/%s.gpx"%d['i'], "w") as f:
        f.write( gpx.to_xml())
        
    #add file info to csv
    df_newrow = pd.DataFrame(d, index=[i])
    df_newrow.to_csv('gpxs/info.csv', mode='a', header=False, index=False)

In [3]:
komoot_urls = ["https://www.komoot.com/tour/2338440718"
               

              ]
for k in komoot_urls:
    add_komoot_url_to_collection(k)